In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LOAD LIBRARIES

In [ ]:
import keras
import keras_nlp

# LOAD GEMMA MODEL

In [ ]:
%%time
LLM = keras_nlp.models.GemmaCausalLM.from_preset('gemma_instruct_2b_en')

# READ THE DATA

The Kaggle notebook gets canceled after 12 hours, while the text summarization of all the reviews in *"selected_reviews.csv"* takes longer than that. To address this, I split the data into five dataframes and performed text summarization on each separately. The code to perform the split is as follows.

**Note: I did the separation in a local notebook. The input and output paths need to be adjusted.**

In [ ]:
"""
summarized_review = pd.read_csv("path/to/selected_reviews.csv")

# Split the DataFrame into smaller DataFrames with approximately 115 rows each
dfs = [summarized_review[i:i + 115] for i in range(0, len(summarized_review), 115)]

# Check the number of DataFrames created
number_of_dfs = len(dfs)
print(f"Number of DataFrames created: {number_of_dfs}")

# Print the shape of each DataFrame
for index, smaller_df in enumerate(dfs):
    print(f"DataFrame {index + 1} shape: {smaller_df.shape}")

# Create the output directory if it doesn't exist
output_dir = "output_kaggleXnotebook/separate dataframes"

# Save each DataFrame to a CSV file in the specified directory
for index, smaller_df in enumerate(dfs):
    filename = f'{output_dir}/reviews_df_{index + 1}.csv'  # Create a filename with the output directory
    smaller_df.to_csv(filename, index=False)  # Save to CSV without row index
    print(f"Saved {filename}")


"""

The five dataframes are named *reviews_df_1*, *reviews_df_2*, *reviews_df_3*, *reviews_df_4*, and *reviews_df_5*. Text summarization needs to be performed individually on each dataframe, requiring the notebook to be run five times.

In [ ]:
selected_reviews = pd.read_csv("/kaggle/input/reviews-separated/reviews_df_1.csv")

In [ ]:
print(selected_reviews.shape)
selected_reviews.head()

In [ ]:
selected_reviews.to_csv("selected_reviews.csv", index = False)

# TEXT SUMMARIZATION

In [ ]:
%%time

# Function to summarize the review text
def summarize_review(review_text):
    
    # for 2 reviews and review counts >= 25, it takes
    instruction = "The text will be a review left for a restaurant by a person. Describe some of the dished ordered by the person. Summarize to a food blogger, how the person overall feels about the menu items and the restaurant. Do not use words like I, they, them, he, she while summarizing. Summarize in a general format"

    context = f"\"{review_text}\"" 
    response = ""
    prompt = f"Instruction:\n{instruction}\n\nContext:{context}\n\nResponse:\n{response}"
    #print(prompt)
    
    # Generate the summary using Gemma model
    summarized_review = LLM.generate([prompt], max_length=1800)[0]
    
    # Extract the response part after "Response:"
    response_start = summarized_review.find("Response:") + len("Response:")
    summarized_review = summarized_review[response_start:].strip()
    
    return summarized_review

selected_reviews['summarized_comments'] = selected_reviews['comments'].apply(summarize_review)

# SAVE OUTPUT

In [ ]:
#selected_reviews_eg.to_csv("selected_reviews_eg_summarized.csv", index = False)
selected_reviews.to_csv("selected_reviews_summarized.csv", index = False)

In [ ]:
print("All done")